In [1]:
get_ipython().run_line_magic('pylab', 'inline')

Populating the interactive namespace from numpy and matplotlib


In [2]:
#from tinc import *
import tinc
import os
import numpy as np
import json

In [3]:
#Enlarge the jupyter notebook so that plots are easier to see 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
A = tinc.Parameter("A", "weighting")
A.values = np.linspace(.1,2,3)

B = tinc.Parameter("B", "weighting")
B.values = np.linspace(0,2,3)

kT = tinc.Parameter("kT", "weighting")
kT.values = np.linspace(.01,.1,3)


graph_buffer = tinc.DiskBufferImage("graph", "out.png", "graph_output")

In [5]:
ps = tinc.ParameterSpace("ps")
ps.register_parameters([A, B, kT])
ps.enable_cache()

Validating json with /home/derick/anaconda3/envs/py37/lib/python3.7/site-packages/tinc/tinc_cache_schema.json


In [6]:
def make_graph(A, B, kT):
    
    vasp_scripts_dir = '/home/derick/vasp_scripts/'

    
    #Assumes that this notebook is running in a directory that contains your cluster expansion fits
    #Requires that you have a "training_set.txt" file in the same directory as this notebook
    ce_fit_dir = os.getcwd()
    fitname = 'A_%s_B_%s_kt_%s' %(str(A), str(B), str(kT))
    os.makedirs(fitname, exist_ok=True)
    os.system('cp training_set.txt %s' % fitname)
    
    
    
    #Format template file
    genetic_alg_input_template = os.path.join(vasp_scripts_dir,'templates', 'genetic_algorithm_settings_template.json')
    with open(genetic_alg_input_template) as f:
      genetic_alg_settings = json.load(f)
      f.close()
        
    genetic_alg_settings['problem_specs']['weight']['kwargs']['A'] = A
    genetic_alg_settings['problem_specs']['weight']['kwargs']['B'] = B
    genetic_alg_settings['problem_specs']['weight']['kwargs']['kT'] = kT

    
    
    with open(os.path.join(fitname, 'genetic_alg_settings.json'), 'w') as settings_file:
        json.dump( genetic_alg_settings, settings_file, indent=4)
        settings_file.close()
    
    
    
    #Run the fit
    os.chdir(fitname)
    print("Removing old data for individual 0")
    os.system('rm check.0; rm checkhull_genetic_alg_settings_0_*; rm genetic_alg_settings_*')
    print("Running new fit")
    os.system('casm-learn -s genetic_alg_settings.json > fit.out')
    print("Writing data for individual 0")
    os.system('casm-learn -s genetic_alg_settings.json --checkhull --indiv 0 > check.0')
    
    
    
    #Plot the fit results
    os.system('python ~/vasp_scripts/plot_clex_hull_data_1.X.py `pwd` 0')
    fit_dir = os.getcwd()
    image_name = fit_dir.split('/')[-3] + '_' + fit_dir.split('/')[-1] + '.png'        
    image_path = os.path.join(fit_dir,image_name)
    
    #return to the ce_fits directory    
    os.chdir('../')
    
    image_file = open(image_path, 'rb')
    return image_file.read()


def cb(value):
    print(value)

def value_changed(value):
    imagedata = ps.run_process(make_graph)
    graph_buffer.data = imagedata

A.register_callback(value_changed)
B.register_callback(value_changed)
kT.register_callback(value_changed)
kT.register_callback(cb)

In [7]:
print(A.values,B.values,kT.values)

[0.1  1.05 2.  ] [0. 1. 2.] [0.01  0.055 0.1  ]


In [8]:
ps.sweep(make_graph)

#currently not using this display approach
from ipywidgets import GridspecLayout
grid = GridspecLayout(3, 2, height='300px')
grid[:, 1] = graph_buffer.interactive_widget()
grid[0, 0] = A.interactive_widget()
grid[1, 0] = B.interactive_widget()
grid[2, 0] = kT.interactive_widget()
grid


In [9]:
from ipywidgets import Button, HBox, VBox

box = VBox([A.interactive_widget(), B.interactive_widget(), kT.interactive_widget(), graph_buffer.interactive_widget()])
graph_buffer._interactive_widget.width = "100%"
box